# 0. Préparation de l'environnement de travail

Pour le bon déroulé du projet, on installe les packages suivants : 
- requests : pour la récupération des données par API
- pandas : pour la création et la manipulation de tableaux de données
- zipfile : pour ouvrir, lire ou extraire des données compressées
- os : pour interagir avec le système de fichiers

In [2]:
import requests
import pandas as pd
import zipfile
import os

# 1. Importation et préparation des données

Des API étaient disponibles pour l'ensemble de nos jeux de données, mais nous avons préféré les télécharger en brut et les mettre en cache local sur S3 pour travailler à partir de là, afin de favoriser la reproductibilité du projet. 

## a. Infrastructures sportives et culturelles

### i. Données

Dans un premier temps, on récupère les données relatives aux infrastructures sportives et culturelles en France. L'Insee (sur **[insee.fr](https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_BPE_SPORT_CULTURE)**) propose une base d'équipements et de service accessibles à la population au niveau communal, intercommunal, départemental, régional... 

> Nous nous intéresserons au **niveau communal**, dans l'optique de faire de même pour les résultats aux élections présidentielles. 

Ce jeu de données est valide pour le **1er janvier 2024**, moins de deux ans après les élections présidentielles. L'écart de temps est donc faible étant donné le temps de construction et la durée de vie en général des infrastructures étudiées, d'où la pertinence de cette base de données. 

<span style="color:red">Le fichier csv téléchargé a été stocké sur notre espace S3 personnel, il faudra probablement changer le chemin d'accès pour y accéder. </span>

In [3]:
path_infrastructures_s3 = "s3://clemencelucas/projet_python_vote_rn/data/raw/DS_BPE_SPORT_CULTURE_2024_data.csv"
df_infrastructures = pd.read_csv(path_infrastructures_s3, sep=';') 

/tmp/ipykernel_11999/537846119.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infrastructures = pd.read_csv(path_infrastructures_s3, sep=';')


Pour vérifier que le data frame est bien chargé et pour avoir un premier aperçu des données, on fait quelques vérifications. 

In [4]:
df_infrastructures.head()

,GEO,GEO_OBJECT,FACILITY_DOM,FACILITY_SDOM,FACILITY_TYPE,INDOOR,LIGHTED,PRACTICE_AREA_ACCESSIBILITY,SANITARY_ACCESSIBILITY,LOCKER_ROOM_ACCESSIBILITY,FREE_ACCESS,SHOWER,SANITARY,SEASONAL_OPENING,MULTIPLEX_CINEMA,ERP_CATEGORY,BPE_MEASURE,TIME_PERIOD,OBS_VALUE
0,69,DEP,F,F1,F101,1,1,0,0,_U,_U,_U,1,0,_Z,2,FACILITIES,2024,1
1,12,DEP,F,F1,F101,0,0,0,0,1,_U,_U,1,1,_Z,3,FACILITIES,2024,1
2,59,DEP,F,F1,F101,1,1,0,0,0,0,1,1,0,_Z,1,FACILITIES,2024,1
3,18,DEP,F,F1,F101,1,1,1,0,0,0,1,1,1,_Z,3,FACILITIES,2024,1
4,45,DEP,F,F1,F101,1,1,1,1,1,_U,_U,1,0,_Z,3,FACILITIES,2024,1


In [5]:
df_infrastructures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351761 entries, 0 to 1351760
Data columns (total 19 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   GEO                          1351761 non-null  object
 1   GEO_OBJECT                   1351761 non-null  object
 2   FACILITY_DOM                 1351761 non-null  object
 3   FACILITY_SDOM                1351761 non-null  object
 4   FACILITY_TYPE                1351761 non-null  object
 5   INDOOR                       1351761 non-null  object
 6   LIGHTED                      1351761 non-null  object
 7   PRACTICE_AREA_ACCESSIBILITY  1351761 non-null  object
 8   SANITARY_ACCESSIBILITY       1351761 non-null  object
 9   LOCKER_ROOM_ACCESSIBILITY    1351761 non-null  object
 10  FREE_ACCESS                  1351761 non-null  object
 11  SHOWER                       1351761 non-null  object
 12  SANITARY                     1351761 non-null  object
 1

On remarque en premier lieu qu'il n'y a pas d'informations manquantes car, pour chaque colonne, on a 1 351 761 non-null correspondant à l'entièreté des lignes. On n'aura de cette façon pas besoin de se préoccuper d'un problème de valeurs manquantes. 

> **Remarque** : Aucune case n'est vide, mais certaines comportent un code signifiant "Indéterminé" (_U) ou "Sans objet" (_Z).

On observe également que panda interprète la quasi totalité des colonnes comme des types *object* donc on lui indique quels sont les types des colonnes qui nous intéressent. 
- GEO : elle comporte un code, une clé qui permet d'identifier une zone géographique. On pourrait penser qu'il s'agit forcément d'un entier, mais les départements Corses sont 2A et 2B et comportent une lettre, et il arrive que des communes aient un code commençant par 0. Il vaut donc mieux considérer cette colonne comme étant de type *string*
- GEO_OBJECT : elle indique le niveau géographique de la ligne (COM pour commune par exemple). Elle est de type *string*. 
- FACILITY_DOM, FACILITY_SDOM, FACILITY_TYPE : elles indiquent la classification des types d'équipements à différents niveaux (catégories, sous-catégories et types). Ce sont des codes de types *string*. 
- BPE_MEASURE : elle est un indicateur d'intérêt de type *string* (A COMPLETER)
- OBS_VALUE : elle indique le nombre d'équipement décrit dans la ligne par les colonnes précédentes présents dans la zone géographique. Elle est de type *int*, mais panda l'a déjà repéré donc pas besoin de faire de modifications. 

Les autres colonnes concernent un degré de précision quant aux équipements que nous considérons trop élevé pour être pertinent (s'il existe une zone illuminée par exemple avec la variable LIGHTED), donc nous les laissons de côté. 

> **Remarque** TIME_PERIOD indique la période d'observation, lors de l'enquête. En l'occurrence, dans ce jeu de données, cette variable est inutile puisqu'elle contient une unique modalité : 2024. 

In [16]:
#vérification que TIME_PERIOD n'a qu'une valeur 
df_infrastructures["TIME_PERIOD"].value_counts()

TIME_PERIOD
2024    1351761
Name: count, dtype: int64

In [13]:
df_infrastructures_typed = df_infrastructures.copy() #on fait une copie pour garder quelque part les données originales
df_infrastructures_typed["GEO"] = df_infrastructures_typed["GEO"].astype(str)
df_infrastructures_typed["GEO_OBJECT"] = df_infrastructures_typed["GEO_OBJECT"].astype(str)
df_infrastructures_typed["FACILITY_DOM"] = df_infrastructures_typed["FACILITY_DOM"].astype(str)
df_infrastructures_typed["FACILITY_SDOM"] = df_infrastructures_typed["FACILITY_SDOM"].astype(str)
df_infrastructures_typed["FACILITY_TYPE"] = df_infrastructures_typed["FACILITY_TYPE"].astype(str)
df_infrastructures_typed["BPE_MEASURE"] = df_infrastructures_typed["BPE_MEASURE"].astype(str)

### ii. Métadonnées

Les métadonnées seront utiles pour avoir des résultats lisibles. Les informations présentes dans le document data comportent essentiellement des codes (F103 pour Tennis par exemple), probablement pour prendre moins d'espace de stockage. Les informations présentes dans le document metadata permettront de clarifier ces clés d'identification. 

In [ ]:
path_infrastructures_meta_s3 = "s3://clemencelucas/projet_python_vote_rn/data/raw/DS_BPE_SPORT_CULTURE_2024_metadata.csv"
df_infrastructures_meta = pd.read_csv(path_infrastructures_meta_s3, sep=';') 

In [ ]:
df_infrastructures_meta.head(20)

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD
0,BPE_MEASURE,Mesure BPE,FACILITIES,Nombre d'équipements
1,BPE_MEASURE,Mesure BPE,PLAYGROUNDS,Nombre d'aires de pratiques sportives d'un mêm...
2,BPE_MEASURE,Mesure BPE,ROOMS,Nombre de salles de cinéma ou théâtre
3,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,1,Au-dessus de 1500 personnes
4,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,3,De 301 à 700 Personnes
5,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,2,De 701 à 1500 personnes
6,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,4,De 101 à 300 personnes
7,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,5,Inférieur ou égal à 100 personnes
8,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,_U,Indéterminé
9,ERP_CATEGORY,Catégorie d’établissement recevant du public (...,_Z,Sans objet


In [10]:
df_infrastructures_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35722 entries, 0 to 35721
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   COD_VAR  35722 non-null  object
 1   LIB_VAR  35722 non-null  object
 2   COD_MOD  35722 non-null  object
 3   LIB_MOD  35722 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


Comme pour le document data, d'une part on n'a pas de valeurs nulles et d'autre part panda considère les 4 colonnes du fichier des métadonnées comme étant de type *object*, alors qu'elles sont toutes de type *string*. On les modifie donc, de la même manière en créant une copie du fichier qu'on modifie. 

In [15]:
df_infrastructures_meta_typed = df_infrastructures_meta.copy() #on fait une copie pour garder quelque part les données originales
df_infrastructures_meta_typed["COD_VAR"] = df_infrastructures_meta_typed["COD_VAR"].astype(str)
df_infrastructures_meta_typed["LIB_VAR"] = df_infrastructures_meta_typed["LIB_VAR"].astype(str)
df_infrastructures_meta_typed["COD_MOD"] = df_infrastructures_meta_typed["COD_MOD"].astype(str)
df_infrastructures_meta_typed["LIB_MOD"] = df_infrastructures_meta_typed["LIB_MOD"].astype(str)

### iii. Analyse exploratoire des données

Dans un premier temps, il faut appréhender au mieux la base de données pour récupérer uniquement les informations utiles. En particulier, pour le plan géographique, on gardera uniquement ce qui concerne les communes. 

In [17]:
df_infrastructures_typed["GEO_OBJECT"].value_counts()

GEO_OBJECT
COM        352946
BV2022     179312
EPCI       168485
UU2020     128029
ARR        122801
ZE2020     118845
AAV2020    113880
DEP         85346
REG         45404
FRANCE      33512
ARM          3201
Name: count, dtype: int64

In [25]:
df_infrastructures_meta_typed["COD_VAR"].value_counts()


COD_VAR
GEO                            35630
FACILITY_TYPE                     37
ERP_CATEGORY                       7
SANITARY_ACCESSIBILITY             4
INDOOR                             4
LIGHTED                            4
LOCKER_ROOM_ACCESSIBILITY          4
MULTIPLEX_CINEMA                   4
FREE_ACCESS                        4
SHOWER                             4
SEASONAL_OPENING                   4
SANITARY                           4
PRACTICE_AREA_ACCESSIBILITY        4
BPE_MEASURE                        3
FACILITY_SDOM                      3
FACILITY_DOM                       1
TIME_PERIOD                        1
Name: count, dtype: int64

Les métadonnées ne renseignent rien sur la variable GEO_OBJECT, donc on ne peut que déduire leur sens : 
- AAV2020 : Aire d'Attraction des Villes en 2020
- ARR : Arrondissement 
- ARM : Arrondissement Municipal 
- BV2022 : Bassin de Vie en 2022
- COM : Commune 
- DEP : Département 
- EPCI : Etablissement Public de Coopération Intercommunal 
- FRANCE : France entière (/!\ avec ou sans outre-mer ?)
- REG : Région
- UU2020 : Unité Urbaine en 2020
- ZE2020 : Zone d'emploi en 2020

<span style="color:red"> regarder si les outre-mer sont inclus, et regarder quelques valeurs en filtrant avec GEO_OBJECT pour vérifier la cohérence + voir pour les arrondissements s'ils sont considérés comme des communes ou non </span>

## b. Elections présidentielles

Afin d'étudier le vote RN dans les communes françaises, nous avons choisi de nous concentrer sur les résultats à la présidentielle 2022 du premier tour, le 10 avril 2022. Nous avions initialement pensé à étudier les résultats au premier tour des législatives 2024, plus récents, mais avons réalisé que bon nombre de circonscriptions n'avaient pas de candidats RN, ce qui aurait pû créer un biais. Avec les présidentielles, nous avions l'assurance d'obtenir des résultats pour chaque commune. 

Entre le premier et le second tour de la présidentielle, nous avons naturellement porté notre attention sur les résultats du premier tour, puisqu'il y avait plus de candidats. 

> Remarque : peut-être détailler un peu plus

Les résultats des élections en France sont disponibles sur **[data.gouv.fr](https://www.data.gouv.fr/datasets/election-presidentielle-des-10-et-24-avril-2022-resultats-definitifs-du-1er-tour/)**. 
Malheureusement, le niveau communal n'est pas proposé, donc nous avons choisi de télécharger le niveau **sous-communal** dans l'optique d'agréger les résultats par la suite. 

In [ ]:
url_elect = "https://www.data.gouv.fr/api/1/datasets/r/6d9b33e5-667d-4c3e-9a0b-5fdf5baac708"
path_elect = "/home/onyxia/work/Projet-Python-S3/code/results_elect.xlsx"

# Téléchargement du fichier Excel
response = requests.get(url_elect)
response.raise_for_status()

with open(path_elect, "wb") as f:
    f.write(response.content)

# Lecture du fichier Excel
df_extrait = pd.read_excel(path_elect, sheet_name=0)

print("Le dataframe df_extrait peut être traité comme un data frame classique avec pandas")

print("Informations sur le dataframe df_extrait qui est notre df initial")
print(df_extrait.columns)
print(len(df_extrait))


# la colonne unamed 53 correspond au nombre de voix de JLF, la colonne unamed 67 correspond à celui de marine le pen
liste_var = ['Code du département', 
    'Libellé du département', 
    'Code de la commune', 
    'Libellé de la commune',
    'Inscrits', 
    'Abstentions',
    '% Abs/Ins',
    'Unnamed: 53', 
    'Unnamed: 67']
df_extrait2 = df_extrait[liste_var]
df_extrait3 = df_extrait2.rename(columns={'Unnamed: 53': 'score_rn'})
df_elect = df_extrait3.rename(columns={'Unnamed: 67': 'score_lfi'})
print("Informations sur le dataframe df_elect")
print(df_elect.columns)
print(len(df_elect))
print(df_elect.head(4))

2) Préparation des données infrastructures
Dans un second temps, nous devons préparer le dataframe des infrastructures. Nous allons le passer d'un format long à un format large, autrement dit, au lieu d'avoir une ligne par infrastructure, nous aurons une ligne par ville et une colonne par infrastructure.

In [ ]:
print("Informations sur le dataframe df_infrastruct")
print(df_infrastruct.columns)
print(df_infrastruct.size)
print(df_infrastruct.head())

In [ ]:
df_infra_large = (
    df_infrastruct
    .query("TIME_PERIOD == 2024")
    .groupby(["GEO", "FACILITY_TYPE"], as_index=False)["OBS_VALUE"]
    .sum()
    .pivot(index="GEO", columns="FACILITY_TYPE", values="OBS_VALUE")
    .fillna(0)
    .reset_index()
)


In [ ]:
print("Vérifications sur le dataframe df_infra_large")
print(df_infra_large.columns)
print(df_infra_large.size)
print(df_infra_large.head())

# 2. Statistiques descriptives 

Cette partie a pour but de vérifier que les bases sont propres et utilisables correctement. Les statistiques descriptives les statistiques descriptives permettront d'être sûres qu'il n'y a pas de données en double, de données manquantes ou de données aberrantes. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# a. Elections présidentielles

Nous allons représenter les résultats aux élections présidentielles des deux partis choisis. Pour ce faire, dans un premier temps, il faut calculer 

In [ ]:
print("Le score moyen par commune du RN était")
print(df_elect["score_rn"].mean())
print(df_elect["score_rn"].median())
print("Le score moyen par commune de LFI était")
print(df_elect["score_lfi"].mean())
print(df_elect["score_lfi"].median())

Si on compare aux scores nationaux, on obtient des chiffres assez proches (Mme Le Pen ayant obtenu 23,15 % des voix au premier tour). La différence entre les deux chiffres peut être due au fait que nous avons fait une moyenne par commune sans pondérer par le nombre d'habitants, ce qui fait que les petites communes, plutôt en campagne et qui votent plus largement pour le RN sur sur-représentées. 

In [ ]:
# Préparation du code département : il faut le formater pour qu'il corresponde aux données des fonds de carte

df_elect['Code du département'] = (
    df_elect['Code du département']
    .astype(str)
    .str.zfill(2)
)

# Agrégation par département

# Moyenne des scores communaux

df_dep = (
    df_elect
    .groupby(['Code du département', 'Libellé du département'], as_index=False)
    .agg({
        'score_rn': 'mean',
        'score_lfi': 'mean'
    })
)


# Chargement du fond de carte départements

gdf_dep = gpd.read_file(
    "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
)

gdf_dep['code'] = gdf_dep['code'].astype(str).str.zfill(2)

# Fusion de la carte avec les données

carte_dep = gdf_dep.merge(
    df_dep,
    left_on='code',
    right_on='Code du département',
    how='left'
)

# Carte RN par département

fig, ax = plt.subplots(figsize=(10, 10))

carte_dep.plot(
    column='score_rn',
    cmap='Blues',
    legend=True,
    edgecolor='black',
    ax=ax
)

ax.set_title("Score Mme Le Pen par département (1er tour 2022)")
ax.axis('off')
plt.show()

# Carte LFI par département

fig, ax = plt.subplots(figsize=(10, 10))

carte_dep.plot(
    column='score_lfi',
    cmap='Reds',
    legend=True,
    edgecolor='black',
    ax=ax
)

ax.set_title("Score M. Mélenchon par département (1er tour 2022)")
ax.axis('off')
plt.show()


Les cartes suivantes sont cohérentes avec les statistiques publiées à propos des élections. Cependant, nous rappelons qu'ici, notre variable d'intérêt est la commune, c'est pour cela que nous n'avons pas pondéré par le nombre d'habitants donc les résultats publiés en ligne. Toutefois, nous ne voyons pas d'incohérence majeure.